In [1]:
import numpy as np
import scipy as sp
from psitools.tanhsinh import TanhSinh

Create integrator. By default, go for $10^{-15}$ maximum precision and 12 levels (i.e. minimum step size $2^{-12}$).

In [2]:
integrator = TanhSinh()

Smooth function, exact result $\int_0^1 x\log(1+x)dx = \frac{1}{4}$:

In [3]:
f = lambda x: x*np.log(1 + x)
print(np.abs(0.25 - integrator.integrate(f, 0, 1)))

9.992007221626409e-16


Custom tolerance, should be larger than maximum precision:

In [4]:
print(np.abs(0.25 - integrator.integrate(f, 0, 1, tol=1.0e-4)))

3.944343737341538e-05


Function with difficult endpoint: $\int_0^1 \sqrt{x} \log x dx = -\frac{4}{9}$:

In [5]:
f = lambda x: np.sqrt(x)*np.log(x)
print(np.abs(-4./9. - integrator.integrate(f, 0, 1)))

0.0


Finally, function with more problematic endpoint: $\int_0^1 \log^2 x = 2$:

In [6]:
f = lambda x: np.log(x)**2
print(np.abs(2.0 - integrator.integrate(f, 0, 1)))

1.3014034294656085e-12


Note that in this case, the requested tolerance is not reached. This is where quadpack does better:

In [7]:
print(np.abs(2.0 - sp.integrate.quad(f, 0, 1)[0]))

8.881784197001252e-16


However, we need integrals like $\int_0^1 \frac{\epsilon}{x^2+\epsilon^2} dx = \tan^{-1}\left(\frac{1}{\epsilon}\right)$ for $\epsilon \sim 10^{-8}$. This is where quadpack fails miserably:

In [8]:
eps = 1.0e-8
f = lambda x: eps/(x**2 + eps**2)
print(sp.integrate.quad(f, 0, 1))

(-1.0000000000061728e-08, 8.540600721962658e-14)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  This is separate from the ipykernel package so we can avoid doing imports until


Fortunately, tanh-sinh does reasonably well. Note that the required tolerance of $10^{-15}$ is not reached, but the result is useable, unlike the quadpack result. 

In [9]:
print(integrator.integrate(f, 0, 1), np.abs(np.arctan(1/eps) - integrator.integrate(f, 0, 1)))

1.5707962136196771 1.0317521947911246e-07
